In [3]:
from math import pi, sqrt
import numpy as np

# m = 56.0 / 1000.0
# L = 2.54 * 34.0 / 100.0
# I = m * L**2 / 3.0

# def calc_g_physical_pendulum(t_5_cycles: float) -> float:
#     T = t_5_cycles / 5.0
#     g = T**2 * m * L / (4 * pi**2 * I)
#     return g

def calc_g_point_mass(t_5_cycles: float, length: float) -> float:
    period = t_5_cycles / 5.0
    angular_freq = (2 * pi) / period
    g = angular_freq**2 * length
    return g

def calc_omega_obs(times: np.ndarray, periods_measured):
    period = np.mean(times) / periods_measured
    return (2 * pi) / period

def calc_omega_0(times: np.ndarray, periods_measured, length):
    period = np.mean(times) / periods_measured
    omega_0 = (2 * pi) / period
    return omega_0

def calc_damping(omega_obs_1, omega_obs_2, m_1, m_2):
    return np.sqrt((omega_obs_2**2 - omega_obs_1**2) / ( 1.0/(4.0*m_2**2) - 1.0/(4.0*m_1**2) ))

# For first mass
m_1 = 112.0 / 1000.0
l_m_1 = 188.75 * 2.54 / 100.0

t_5_m_1 = np.array([
    22.037,
    21.943,
    22.053
])
t_std_1 = np.std(t_5_m_1)
omega_obs_m_1 = calc_omega_obs(t_5_m_1, 5.0)

# For second mass
m_2 = 38.0 / 1000.0
l_m_2 = 186.75 * 2.54 / 100.0
t_5_m_2 = np.array([
    21.562,
    21.808,
    21.670
])
t_std_2 = np.std(t_5_m_2)
omega_obs_m_2 = calc_omega_obs(t_5_m_2, 5.0)

# Calculate damping then g
b = calc_damping(omega_obs_m_1, omega_obs_m_2, m_1, m_2)

def calc_g_with_damping(omega_obs, b, m, l):
    return l * (omega_obs**2 + b**2 / (4.0*m**2))

g = calc_g_with_damping(omega_obs_m_1, b, m_1, l_m_1)
print(g)

9.805635060554328


In [4]:
# Propogate error
uomega_obs_1 = t_std_1 / np.mean(t_5_m_1)
uomega_obs_2 = t_std_2 / np.mean(t_5_m_2)
ul = 0.1 / 100.0 / l_m_1
um_1 = 1.0 / m_1 / 1000.0
um_2 = 1.0 / m_2 / 1000.0
ub = 0.5 * ((2*uomega_obs_1 + 2*uomega_obs_2)**2 + (2*um_1 + 2*um_2)**2)
uf = sqrt((2*ub)**2 + (2*um_1)**2)
ug = sqrt(ul**2 + (2*uomega_obs_1 + uf)**2)
print(ug)

0.022996774122681515


In [5]:
from err import Measurement as M

def calc_omega_obs(t):
    period = t / 5.0
    return (2 * pi) / period

def calc_damping(omega_obs_1, omega_obs_2, m_1, m_2):
    return (omega_obs_2**2 - omega_obs_1**2) / ( 1.0/(4.0*m_2**2) - 1.0/(4.0*m_1**2) )**0.5

m_1 = M(m_1, um_1)
m_2 = M(m_2, um_2)
l_1 = M(l_m_1, ul)
l_2 = M(l_m_2, ul)
t_1 = M.from_list(t_5_m_1)
t_2 = M.from_list(t_5_m_2)
omega_obs_1 = calc_omega_obs(t_1)
omega_obs_2 = calc_omega_obs(t_2)
b = calc_damping(omega_obs_1, omega_obs_2, m_1, m_2)
g = calc_g_with_damping(omega_obs_1, b, m_1, l_1)
print(g)

9.769±0.082
